In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("test_lib_from_postgres_to_clickhouse")
    .config(
        "spark.jars.packages",
        ",".join([
            # clickhouse drivers
            "com.clickhouse:clickhouse-jdbc:0.7.2",
            # postgres drivers
            "org.postgresql:postgresql:42.7.3"
        ])
    )
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.clickhouse#clickhouse-jdbc added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2ee21ce8-1954-43e0-a30b-4dc9a23dc2c7;1.0
	confs: [default]
	found com.clickhouse#clickhouse-jdbc;0.7.2 in central
	found com.clickhouse#clickhouse-client;0.7.2 in central
	found com.clickhouse#clickhouse-data;0.7.2 in central
	found com.clickhouse#clickhouse-http-client;0.7.2 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.httpcomponents.client5#httpclient5;5.2.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2.1 in central
	found org.apache.commons#commons-compress;1.27.1 in central
	found commons-codec#commons-codec;1.17.1 in central
	found commons-io#commons-io;2.16.1 in central
	found org.apache.commons

In [2]:
from data_transfer_lib import Postgres, ClickHouse, Reader, Writer

In [3]:
postgres_conn = Postgres(
    host="postgres",
    user="postgres_user",
    password="postgres_password",
    database="postgres",
    port=5432,
    spark=spark
)

reader = Reader(
    connection=postgres_conn,
    db_name="postgres",
    table_name="test_data"
)

df = reader.start()

Validation succeeded


In [4]:
df.show()

+---+-------+-----+
| id|   name|value|
+---+-------+-----+
|  1|  Alice|   10|
|  2|    Bob|   20|
|  3|Charlie|   30|
|  4|  Alice|   10|
|  5|    Bob|   20|
|  6|Charlie|   30|
+---+-------+-----+



In [5]:
clickhouse_conn = ClickHouse(
    host="clickhouse-server",
    user="default",
    password="1234qwe",
    database="default",
    spark=spark
)

In [6]:
writer = Writer(
    connection=clickhouse_conn,
    db_name="default",
    table_name="test_table",
    if_exists=True
)

{'id': 'Int32', 'name': 'Nullable(String)', 'value': 'Nullable(Int32)'}


In [7]:
writer.start(
    df=df,
    mode="append",
    batch_size=10000,
    num_partitions=5
)

Валидация пройдена: схемы совместимы


25/11/19 16:23:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/19 16:23:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/19 16:23:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/19 16:23:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/19 16:23:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/19 16:23:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQ

In [16]:

# Чтение из Postgres
reader = Reader(
    connection=postgres_conn,
    db_name="postgres",
    table_name="test_data"
)
df = reader.start()


# Загрузка в ClickHouse
writer = Writer(
    connection=clickhouse_conn,
    db_name="analytics",
    table_name="users",
    if_exists=True
)
uploader.start(df=df)

Инициализирован Postgres
Валидация параметров подключения
Инициализирован ClickHouse
Валидация параметров подключения
Инициализирован Reader
Вызван метод Postgres.get_table_schema для postgres.test_data
{'id': 'integer', 'name': 'text', 'value': 'integer'}
Валидация пройдена: все типы данных поддерживаются
Получить свойства подключения


Py4JJavaError: An error occurred while calling o89.load.
: org.postgresql.util.PSQLException: ERROR: relation "postgres.test_data" does not exist
  Position: 15
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2725)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2412)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:371)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:502)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:419)
	at org.postgresql.jdbc.PgPreparedStatement.executeWithFlags(PgPreparedStatement.java:194)
	at org.postgresql.jdbc.PgPreparedStatement.executeQuery(PgPreparedStatement.java:137)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:68)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation

In [ ]:
print(postgres_conn.get_jdbc_url())
print(clickhouse_conn.get_jdbc_url())

In [14]:
# Чтение из Postgres
reader = Reader(
    connection=postgres_conn,
    db_name="postgres",
    table_name="test_data"
)

Инициализирован Reader
Вызван метод Postgres.get_table_schema для postgres.test_data
{'id': 'integer', 'name': 'text', 'value': 'integer'}
Валидация пройдена: все типы данных поддерживаются


In [6]:
reader.source_schema

{'id': 'integer', 'name': 'text', 'value': 'integer'}

In [7]:
reader.connection.get_jdbc_url()

'jdbc:postgresql://postgres:5432/postgres'

In [10]:
full_table_name = f"{reader.db_name}.{reader.table_name}" if reader.db_name else self.table_name

r = (
    spark.read.format("jdbc")
    .option("dbtable", full_table_name)
)

SyntaxError: incomplete input (2208657447.py, line 1)

In [15]:
df = reader.start()

Получить свойства подключения


Py4JJavaError: An error occurred while calling o71.load.
: org.postgresql.util.PSQLException: ERROR: relation "postgres.test_data" does not exist
  Position: 15
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2725)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2412)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:371)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:502)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:419)
	at org.postgresql.jdbc.PgPreparedStatement.executeWithFlags(PgPreparedStatement.java:194)
	at org.postgresql.jdbc.PgPreparedStatement.executeQuery(PgPreparedStatement.java:137)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:68)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation

In [7]:
clickhouse_conn = ClickHouse(
    host="clickhouse-server",
    user="default",
    password="1234qwe",
    database="default",
    spark=spark
)

Инициализирован ClickHouse
Валидация параметров подключения


In [1]:




df = reader.start()

# Загрузка в ClickHouse
uploader = Writer(
    connection=clickhouse_conn,
    db_name="analytics",
    table_name="users",
    if_exists=True
)
uploader.start(df=df)

ModuleNotFoundError: No module named 'data_transfer_lib'